In [1]:
!pip install Sastrawi


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display

In [2]:
df = pd.read_excel("data/hasilCleaning.xlsx")
df

,Message
0,NaN
1,siap siap buat isi full tank bro
2,promo tebar undian apa masih bisa ikutan
3,okaaay siaaapp noted
4,buat pengguna baru aja gak sih
...,...
7064,wowww ada promo spesial akhir tahun
7065,kalo gue yang pake bakal keren gak
7066,min abis tukar nanti ambil dimana hadiahnya
7067,loh iya toh besok tahun baruan


In [3]:
df = df.dropna()

In [4]:
df.shape

(6949, 1)

## Filter Tokens

In [5]:
# Definisi fungsi filter_tokens_by_length
def filter_tokens_by_length(dataframe, column, min_words, max_words):
    # Tokenisasi kata
    words_count = dataframe[column].astype(str).apply(lambda x: len(x.split()))
    # Membuat filter untuk jumlah kata
    mask = (words_count >= min_words) & (words_count <= max_words)
    # Mengaplikasikan filter ke DataFrame
    filtered_df = dataframe[mask]
    return filtered_df

# Menggunakan filter_tokens_by_length untuk mendapatkan baris dengan jumlah kata antara 3 dan 50
min_words = 3
max_words = 50
df = filter_tokens_by_length(df, 'Message', min_words, max_words)

In [6]:
df

,Message
1,siap siap buat isi full tank bro
2,promo tebar undian apa masih bisa ikutan
3,okaaay siaaapp noted
4,buat pengguna baru aja gak sih
5,ya haaayyuukkk gaskeun
...,...
7064,wowww ada promo spesial akhir tahun
7065,kalo gue yang pake bakal keren gak
7066,min abis tukar nanti ambil dimana hadiahnya
7067,loh iya toh besok tahun baruan


## Normalisasi

In [7]:
# Normalisasi
norm = {' mikir2 ':' mikir ', ' siaaapp ':' siap ', 'okaaay ':'oke ', 'udh ':'sudah ','ga ':'tidak ',' gaskeun':' ayo ', 'wowww ':'wow ', ' haaayyuukkk ':' ayo ', ' yg ':' yang ', ' udh ':' udah ', 'wkwk ':' ', ' min ':' kak ', ' malem ':' malam', ' malem2 ':' malam ', ' sm ':' sama ', ' dy ':' dia ', ' lg ':' lagi ', ' skrg ':' sekarang ', ' ddpn ':' didepan ', ' makasi ':' makasih ', ' pertamaz ':' pertamax ', ' jg ':' juga ', ' donk ':' dong ', ' ikutann ':' ikutan ', ' banyakk ':' banyak ', ' twt ':' tweet', 'mantaap ':'mantap ', ' juarak':' juara ', 'daridulu ':'dari dulu ', 'siapp ':'siap ', ' gamau ':' tidak mau ', ' sll ':' selalu ', ' qu ':' aku ', ' krn ':' karena ', ' irii':' iri', ' muluu ':' terus ', 'mada ':'masa ', 'jgn ':'jangan ', ' jgn ':' jangan ', ' muluuu ':' terus ', 'ntar ':'nanti ', ' awtnya':' awetnya', 'gg ':'keren ', ' kerennn':' keren ', ' bisaa ':' bisa ', 'gaaa':'tidak ', " yg ": " yang ", ' nyampe':' sampai', ' nyampe ':' sampai ', ' lu ':' kamu ', ' ikhlaaasss ':' ikhlas ', ' gak ':' tidak ', ' klo ':' kalo ', ' amp ': ' sampai ', ' ga ':' tidak ', ' yaaaa':' ya ', 'betolll ':'betul ', ' kaga ':' tidak ', ' idk ':' tidak tahu ', ' jkt ':' jakarta ', ' lo ':' kamu ', ' bjir ':' ', ' kek ':' seperti ', ' yg ':' yang ', ' utk ':' untuk ', 'kismin ':'miskin ', ' kismin ':' miskin ', ' pd ':' pada ', ' dgn ':' dengan ', ' ituu ':' itu ', ' jg ':' juga ', 'yoi':'iya ', ' yoi ':' iya ', 'org2 ':'orang ', ' tak ':' tidak ', ' kyk ':' seperti ', ' sbg ':' sebagai ', ' anjjjj ':' ', ' bgt ':' banget ', 'km ':'kamu ', ' km ':' kamu', ' byk ':' banyak ', ' lg ':' lagi ', ' mrk ':' mereka ', ' blm ':' belum '}

def normalisasi(str_text):
  for i in norm:
    str_text = str_text.replace(i, norm[i])
  return str_text

df['Message'] = df['Message'].apply(lambda x: normalisasi(x))
df

,Message
1,siap siap buat isi full tank bro
2,promo tebar undian apa masih bisa ikutan
3,oke siap noted
4,buat pengguna baru aja tidak sih
5,ya ayo ayo
...,...
7064,wow ada promo spesial akhir tahun
7065,kalo gue yang pake bakal keren gak
7066,min abis tukar nanti ambil dimana hadiahnya
7067,loh iya toh besok tahun baruan


## Stopwords

In [9]:
# Stopwords
import Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
more_stop_words = []

stop_words = StopWordRemoverFactory().get_stop_words()
stop_words.extend(more_stop_words)

new_array = ArrayDictionary(stop_words)
stop_words_remover_new = StopWordRemover(new_array)

def stopword(str_text):
  str_text = stop_words_remover_new.remove(str_text)
  return str_text

df['Message'] = df['Message'].apply(lambda x: stopword(x))
df.head()

,Message
1,siap siap buat isi full tank bro
2,promo tebar undian apa bisa ikutan
3,oke siap noted
4,buat pengguna baru aja sih
5,ayo ayo


## Tokenize

In [10]:
# Tokenize
tokenized = df['Message'].apply(lambda x:x.split())
tokenized

1                [siap, siap, buat, isi, full, tank, bro]
2               [promo, tebar, undian, apa, bisa, ikutan]
3                                      [oke, siap, noted]
4                        [buat, pengguna, baru, aja, sih]
5                                              [ayo, ayo]
                              ...                        
7064                  [wow, promo, spesial, akhir, tahun]
7065                 [kalo, gue, pake, bakal, keren, gak]
7066                 [min, abis, tukar, ambil, hadiahnya]
7067                     [loh, iya, besok, tahun, baruan]
7068    [tidak, perlu, mikir, deh, mending, buruan, tu...
Name: Message, Length: 6419, dtype: object

## Stemming

In [11]:
# Stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemming(text_cleaning):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  do = []
  for w in text_cleaning:
    dt = stemmer.stem(w)
    do.append(dt)
  d_clean = []
  d_clean = " ".join(do)
  print(d_clean)
  return d_clean

tokenized = tokenized.apply(stemming)

tokenized.to_csv("data/hasil_preprocessingData.csv", index=False)

siap siap buat isi full tank bro
promo tebar undi apa bisa ikut
oke siap noted
buat guna baru aja sih
ayo ayo
tanggung bangettt kak masa cuman dapat sekali doang
min cashback khusus bayar via ovo aja itu dapet arti
lumayan kak tp kudu akumulasi 1 jt 7 hari
min panjang dong jangan per periode 7 hari kl tiap akhir bulan min
reminder kak coz suka lupa
mimin double poin kapan lagi
hmmm suka skip info kak saran kak kl di spbu info2 gin
promonya kaleng2 emang
yaudah emang banyak duit
selamat hari buruh nasional
gue baru aja redeem poin jadi voucher makan
untung gw beli evoucherbesok sekali dah full in 1 jt
banyak promo kayak gin kak gue sukaaa
udah isi bro anto segera donlod my pertamina gihh banyak promonya
gw aja pake supra x 125 2006 always shell super ato minimal pertamax
pertamax sini tetep ngantri beli turbo baru ngantri
lebih irit green 95 d banding pertamax 92
serius nanya fast track tuh semua pertamax cuma pertamax turbo gw jakarta bandung semarang diri beli turbo suruh nyelak cuma 

In [15]:
tokenized

,content
5,"[ongkir, mahal, pajak, aplikasi, gede, biaya, ..."
8,"[aplikasi, jelas, sering, banget, gofood, udah..."
9,"[pas, butuh, malah, setting, lokasi, udah, pak..."
10,"[update, aplikasinya, malah, menyusahkan, pesa..."
11,"[benar2, membantu, cuma, kadang2, apknya, suka..."
...,...
116995,"[sangat, bagus, dipahamidan, membantu]"
116996,"[gercep, kebanyakan, driver, baca, map, jadi, ..."
116997,"[sangat, membantu, promonya, banyak, sukses, s..."
116998,"[makin, kesini, malah, makin, kesana, ongkirny..."
